In [4]:
%load_ext autoreload
%autoreload 2

# add src to sys-path
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append('./src')
from matplotlib.pyplot import figure
from geopy.distance import geodesic
from sklearn.decomposition import PCA

from data_processing.mtb_data_provider import MtbDataProvider
from data_processing.mtb_data_tagger import MtbDataTagger
from data_processing.mtb_classifier import MtbClassifier
from visualization.mtb_visualizer import MtbVisualizer
from data_processing.helpers import Helpers

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, SpectralClustering, MeanShift, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

import geopandas as gpd
from geopandas import GeoDataFrame

from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

SPEED_THRESHOLD = .3
LATITUDE_KEY = 'position_lat'
LONGITUDE_KEY = 'position_long'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
mtb_data_provider = MtbDataProvider()
mtb_data_tagger = MtbDataTagger()
mtb_visualizer = MtbVisualizer()
mtb_classifier = MtbClassifier()

## Data Export

In [86]:
warngau_files=['warngaus2_1', 'warngaus2_2']#'warngaus1_1', 'warngaus1_2', 'warngaus1_3', 
location_based_label_files=['warngaus2_labels', 'warngaus2_labels']
columns = ['SensorAccelerationX_HD', 'SensorAccelerationY_HD', 'SensorAccelerationZ_HD','altitude', 'speed', 'heart_rate', 'SensorHeading', LATITUDE_KEY, LONGITUDE_KEY]
#window_lengths = [50, 100, 150, 200, 300]
#sub_sample_lengths = [25, 50, 75, 100, 150]
window_lengths = [50, 150, 300]
sub_sample_lengths = [25, 75, 150]
nums_clusters = [3, 4, 6, 9, 12]# TODO: ,4,6,8,12,16]
prefix='warngau_s2'

### Classification

In [ ]:
X_neureuth_tmp = data_neureuth
y_neureuth_tmp = osm_difficulty_results

X_neureuth = []
y_neureuth = []

# Ignore Sections without difficulty rating
for i in range(len(y_neureuth_tmp)):
    if y_neureuth_tmp[i] > 0:
        X_neureuth.append(X_neureuth_tmp[i])
        y_neureuth.append(y_neureuth_tmp[i])
        

X_neureuth = np.asarray(X_neureuth)
y_neureuth = np.asarray(y_neureuth)

In [ ]:
classifiers = [
KNeighborsClassifier(3),
#LinearSVC(C=0.025),
DecisionTreeClassifier(max_depth=5, random_state=42),
RandomForestClassifier(max_depth=5, n_estimators=10, random_state=42),
#MLPClassifier(alpha=1, max_iter=20000, random_state=42),
#KMeans(n_clusters=2),
]

names = [
    "Nearest Neighbors", 
#    "Linear SVM", 
    "Decision Tree", 
    "Random Forest", 
#    "Neural Net", 
#   "KMeans",
]

mtb_classifier.run_classification(X_neureuth, y_neureuth, classifiers, names, mtb_data_provider, mtb_visualizer, n_splits=30, clear_outliers=False, print_plots=False)

### CNN

In [ ]:
mtb_classifier.run_cnn_model(X_neureuth, y_neureuth, mtb_data_provider, mtb_visualizer, n_conv_blocks=3)